In [1]:
import pandas as pd
import requests
import os
from google.cloud import bigquery

In [2]:
os.environ["GOOGLE_APPLICATION_CREDENTIALS"] = 'secret.json'

bq_client = bigquery.Client()

[API Docs](https://dev.socrata.com/foundry/data.cityofnewyork.us/ic3t-wcy2)  
[OpenData](https://data.cityofnewyork.us/Housing-Development/DOB-Job-Application-Filings/ic3t-wcy2)  
[Metadata](https://docs.google.com/spreadsheets/d/1TGe-YvR-rqV3sTm2DVl9icoiypo3ug-GDdP1ULdo1_s/edit#gid=169410431)

In [3]:
ACRIS=f'https://data.cityofnewyork.us/resource/bnx9-e6tj.json?$'

In [4]:
acris_count=f'https://data.cityofnewyork.us/resource/bnx9-e6tj.json?$select=count(1) as cnt'
cnt_req=requests.get(acris_count).json()
nrows=int(cnt_req[0]['cnt'])

In [5]:
limit=10**6
offset=0

offs=list(range(0, nrows, limit))

reqs=[f'{ACRIS}limit={limit}&$offset={o}&$order=recorded_datetime desc' for o in offs]

In [6]:
table_id='pluto-panel.real_estate.raw_acris_master2'

job_config = bigquery.LoadJobConfig(
        autodetect=True,
        write_disposition='WRITE_APPEND'

        )

In [8]:
prog=1
for r in reqs:
    print(f'reading {prog} / {len(reqs)}: {r}')
    d=requests.get(r).json()
    print('making dataframe')
    d2=pd.DataFrame(d)
    print('loading to bigquery')
    job = bq_client.load_table_from_dataframe(
            d2, table_id, job_config=job_config)
    job.result()
    prog+=1
    
    

reading 1 / 16: https://data.cityofnewyork.us/resource/bnx9-e6tj.json?$limit=1000000&$offset=0&$order=recorded_datetime desc
making dataframe
loading to bigquery
reading 2 / 16: https://data.cityofnewyork.us/resource/bnx9-e6tj.json?$limit=1000000&$offset=1000000&$order=recorded_datetime desc
making dataframe
loading to bigquery
reading 3 / 16: https://data.cityofnewyork.us/resource/bnx9-e6tj.json?$limit=1000000&$offset=2000000&$order=recorded_datetime desc
making dataframe
loading to bigquery
reading 4 / 16: https://data.cityofnewyork.us/resource/bnx9-e6tj.json?$limit=1000000&$offset=3000000&$order=recorded_datetime desc
making dataframe
loading to bigquery
reading 5 / 16: https://data.cityofnewyork.us/resource/bnx9-e6tj.json?$limit=1000000&$offset=4000000&$order=recorded_datetime desc
making dataframe
loading to bigquery
reading 6 / 16: https://data.cityofnewyork.us/resource/bnx9-e6tj.json?$limit=1000000&$offset=5000000&$order=recorded_datetime desc
making dataframe
loading to bigquer

In [9]:
table = bq_client.get_table(table_id)  # Make an API request.
print(
    "Loaded {} rows and {} columns to {}".format(
        table.num_rows, len(table.schema), table_id
    )
)

Loaded 15600757 rows and 14 columns to pluto-panel.real_estate.raw_acris_master2
